In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d grassknoted/asl-alphabet
! unzip asl-alphabet


In [ ]:
#from tensorflow.keras.applications.ResNet50 import ResNet50

import pandas as pd
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import tensorflow.keras
import cv2
import shutil
import copy

In [ ]:
img_h, img_w = (200,200)
base_model = tensorflow.keras.applications.ResNet50V2(include_top=False, weights='imagenet', input_shape=(200,200,3))
for layer in base_model.layers:
  layer.trainable = False
num_classes = 29
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
predict = Dense(num_classes, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = predict)

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
EPOCHS = 10
BS = 64
aug = ImageDataGenerator(rotation_range=5, zoom_range=0.2, rescale=1./255,
                         width_shift_range=0.2, height_shift_range=0.2,
                         shear_range=0.2,
                         horizontal_flip=False, fill_mode="nearest",validation_split = 0.3)
directory = "/content/asl_alphabet_train/asl_alphabet_train/"
image_generator = aug.flow_from_directory(directory, target_size=(200,200), class_mode = 'categorical', batch_size = BS,subset = 'training')

#test_datagen = ImageDataGenerator(rescale=1./255)
validation_directory = '/content/asl_alphabet_train/asl_alphabet_train'
validation_generator = aug.flow_from_directory(
        validation_directory,
        target_size=(200, 200),
        batch_size=BS,
        class_mode='categorical',
        subset = 'validation')

H = model.fit_generator(image_generator, steps_per_epoch=800, epochs=EPOCHS, validation_data=validation_generator,validation_steps=800 // BS)

In [ ]:
for layer in model.layers:
    layer.trainable = True

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

EPOCHS = 2
BS = 32
aug = ImageDataGenerator(rotation_range=5, zoom_range=0.2, rescale=1./255,
                         width_shift_range=0.2, height_shift_range=0.2,
                         shear_range=0.18,
                         horizontal_flip=False, fill_mode="nearest",validation_split = 0.3)

#directory for image as data
directory = "/content/asl_alphabet_train/asl_alphabet_train/"

image_generator = aug.flow_from_directory(directory, target_size=(200, 200), 
                    class_mode='categorical',batch_size=BS, subset = 'training')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_directory = '/content/asl_alphabet_train/asl_alphabet_train/'
validation_generator = aug.flow_from_directory(
        directory,
        target_size=(200, 200),
        batch_size=BS,
        class_mode='categorical',
        subset = 'validation')

H2 = model.fit_generator(image_generator,steps_per_epoch=2000, epochs=EPOCHS, validation_data=validation_generator,validation_steps=800 // BS)


In [ ]:
model.save("./resnetV2.h5")

H.history

H2.history

In [ ]:
import tensorflow as tf
print(tf.__version__)